# Proposed

- Conv1D layer -> LSTM

# 1. import

In [1]:
from emotion_recognition import utils
from emotion_recognition import features

import numpy as np

import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras import layers
from keras.models import Sequential
from keras import regularizers
from keras.objectives import categorical_crossentropy


import warnings

warnings.filterwarnings(action='ignore') 

Using TensorFlow backend.


# 2. Data preparation

- Setting parameters

In [2]:
# Parameter
time_step = 32
window_size = 1000
overlap_facor = 3
class_size = 4
feature_size = 160
frame_size = 500 # 음성구간 찾기 위한 프레임 크기 

# 0 = neutral, 1 = anger, 2 = happiness, 3 = sadness
emotion = ["neutral","anger","happiness","sadness"]

- Loading filelist 

In [3]:
# 파일 이름 읽어오기
path = "../data/"
text_filename = "filelist_wav.txt"

filelist_wav = []
emotionlist = []

f = open(path+text_filename, 'r')

while True:
    line = f.readline()
    if not line: break
    
    filename, label = line.split()
    
    filelist_wav.append(filename)
    emotionlist.append(label)
    
f.close()

- Splitting the data into trainnnig and test
- For test, subjects 1,6,7,11 
- For training, subjects 2,3,4,5,8,9,10,12,13,14,15,16,17,18,19,20

In [4]:
Training_filename = []
Training_emotionlist = []
Test_filename = []
Test_emotionlist = []

for i in range(len(filelist_wav)):
    if (filelist_wav[i].split("_")[0][1:] == '1'):
        Test_filename.append(filelist_wav[i])
        Test_emotionlist.append(emotionlist[i])
    elif (filelist_wav[i].split("_")[0][1:] == '6'):
        Test_filename.append(filelist_wav[i])
        Test_emotionlist.append(emotionlist[i])
    elif (filelist_wav[i].split("_")[0][1:] == '7'):
        Test_filename.append(filelist_wav[i])
        Test_emotionlist.append(emotionlist[i])
    elif (filelist_wav[i].split("_")[0][1:] == '11'):
        Test_filename.append(filelist_wav[i])
        Test_emotionlist.append(emotionlist[i])
    else:
        Training_filename.append(filelist_wav[i])
        Training_emotionlist.append(emotionlist[i])

In [5]:
print (np.shape(Training_filename))
print (np.shape(Test_filename))

(614,)
(226,)


- Speech data segmentation without unvoice

- Training vector

In [6]:
import math

Training_vector = []
Training_label = []

for ix in range(len(Training_filename)):
    # 파일 읽어오기
    if Training_emotionlist[ix] == "excitement":
        continue
        
    if Training_emotionlist[ix] == "fear":
        continue
        
    print (str(ix)+ "\t" + Training_emotionlist[ix] + "\t" + path+'wav/' + Training_filename[ix])
    
    y,sr = utils.loadwav(path+'wav/' + Training_filename[ix])
    
    temp = emotion.index(Training_emotionlist[ix])    
    label = np.zeros(class_size)
    label[temp] = 1
    
    #파일 전체 길이
    length = len(y)
    
    idx = 0
    while(idx != length):
        #voice 구간 구하기
        IAV_th = utils.get_IAV_threshold(y,length,frame_size)
        th = IAV_th/frame_size*2
        
        start_point, end_point = utils.search_voicearea(y,frame_size,length,idx,th,IAV_th)
        
        if(start_point == -1):
            break
            
        idx = end_point
        
        segment_length = end_point-start_point+1
        
        # 세그먼트 추출
        segment_list = []
        for i in range(start_point,(end_point-window_size), int(window_size/overlap_facor) ):
            y_sub = y[i:i+window_size]
            
            #Normalization
            y_sub = y_sub * 2 / math.pow(2,16)
            segment_list.append(y_sub)
        
        if len(segment_list) < time_step:
            continue
        
        for i in range(0, len(segment_list) - time_step+1, 30):
            temp = segment_list[i:i+time_step]
            Training_vector.append(temp)
            Training_label.append(label)
            
X_train = np.array(Training_vector)
Y_train = np.array(Training_label)

0	sadness	../data/wav/s19_sadness_M_s17.wav
1	sadness	../data/wav/s18_sadness_F_s1.wav
4	anger	../data/wav/s15_anger_F_a11.wav
6	sadness	../data/wav/s19_sadness_M_s8.wav
8	neutral	../data/wav/s18_neutral_F_n14.wav
9	sadness	../data/wav/s20_sadness_F_s20.wav
11	happiness	../data/wav/s3_happiness_M_h10.wav
13	happiness	../data/wav/s5_happiness_M_h8.wav
14	sadness	../data/wav/s8_sadness_M_s17.wav
15	happiness	../data/wav/s5_happiness_M_h16.wav
16	happiness	../data/wav/s4_happiness_M_h10.wav
17	neutral	../data/wav/s9_neutral_M_n10.wav
20	anger	../data/wav/s19_anger_M_a18.wav
21	neutral	../data/wav/s15_neutral_F_n11.wav
22	anger	../data/wav/s14_anger_M_a17.wav
23	neutral	../data/wav/s12_neutral_F_n5.wav
24	neutral	../data/wav/s15_neutral_F_n16.wav
25	happiness	../data/wav/s19_happiness_M_h7.wav
28	happiness	../data/wav/s10_happiness_M_h10.wav
29	happiness	../data/wav/s20_happiness_F_h8.wav
30	happiness	../data/wav/s19_happiness_M_h8.wav
31	sadness	../data/wav/s12_sadness_F_s12.wav
32	neutra

282	happiness	../data/wav/s19_happiness_M_h9.wav
284	neutral	../data/wav/s9_neutral_M_n13.wav
285	happiness	../data/wav/s12_happiness_F_h4.wav
287	anger	../data/wav/s13_anger_F_a12.wav
288	happiness	../data/wav/s10_happiness_M_h14.wav
289	happiness	../data/wav/s2_happiness_M_h18.wav
290	neutral	../data/wav/s19_neutral_M_n13.wav
291	neutral	../data/wav/s13_neutral_F_n13.wav
292	sadness	../data/wav/s15_sadness_F_s5.wav
294	anger	../data/wav/s5_anger_M_a8.wav
296	sadness	../data/wav/s4_sadness_M_s18.wav
298	sadness	../data/wav/s4_sadness_M_s17.wav
299	sadness	../data/wav/s9_sadness_M_s1.wav
301	sadness	../data/wav/s10_sadness_M_s12.wav
302	happiness	../data/wav/s8_happiness_M_h5.wav
303	anger	../data/wav/s15_anger_F_a17.wav
304	neutral	../data/wav/s12_neutral_F_n12.wav
305	sadness	../data/wav/s15_sadness_F_s15.wav
306	anger	../data/wav/s15_anger_F_a16.wav
307	neutral	../data/wav/s14_neutral_M_n20.wav
308	neutral	../data/wav/s14_neutral_M_n13.wav
311	neutral	../data/wav/s9_neutral_M_n2.wav

554	happiness	../data/wav/s15_happiness_F_h13.wav
559	happiness	../data/wav/s3_happiness_M_h1.wav
560	sadness	../data/wav/s9_sadness_M_s19.wav
561	neutral	../data/wav/s12_neutral_F_n8.wav
563	sadness	../data/wav/s2_sadness_M_s18.wav
564	happiness	../data/wav/s9_happiness_M_h16.wav
565	anger	../data/wav/s9_anger_M_a11.wav
566	neutral	../data/wav/s3_neutral_M_n16.wav
567	anger	../data/wav/s19_anger_M_a19.wav
568	sadness	../data/wav/s13_sadness_F_s5.wav
569	sadness	../data/wav/s13_sadness_F_s12.wav
574	sadness	../data/wav/s18_sadness_F_s17.wav
576	neutral	../data/wav/s14_neutral_M_n14.wav
577	happiness	../data/wav/s9_happiness_M_h4.wav
580	neutral	../data/wav/s3_neutral_M_n8.wav
582	happiness	../data/wav/s12_happiness_F_h12.wav
583	sadness	../data/wav/s10_sadness_M_s17.wav
585	sadness	../data/wav/s19_sadness_M_s2.wav
586	sadness	../data/wav/s17_sadness_M_s16.wav
588	anger	../data/wav/s18_anger_F_a16.wav
589	sadness	../data/wav/s4_sadness_M_s14.wav
590	happiness	../data/wav/s19_happiness_M

In [7]:
print (np.shape(X_train),np.shape(Y_train))

(1522, 32, 1000) (1522, 4)


- Test vector

In [8]:
Test_vector = []
Test_label = []

for ix in range(len(Test_filename)):
    # 파일 읽어오기
    if Test_emotionlist[ix] == "excitement":
        continue
        
    if Test_emotionlist[ix] == "fear":
        continue
        
    #print (str(ix)+ "\t" + Test_emotionlist[ix] + "\t" + path+'wav/' + Test_filename[ix])
    
    y,sr = utils.loadwav(path+'wav/' + Test_filename[ix])
    
    temp = emotion.index(Test_emotionlist[ix])    
    label = np.zeros(class_size)
    label[temp] = 1
    
    #파일 전체 길이
    length = len(y)
    
    idx = 0
    while(idx != length):
        #voice 구간 구하기
        IAV_th = utils.get_IAV_threshold(y,length,frame_size)
        th = IAV_th/frame_size*2
        
        start_point, end_point = utils.search_voicearea(y,frame_size,length,idx,th,IAV_th)
        
        if(start_point == -1):
            break
            
        idx = end_point
        
        segment_length = end_point-start_point+1
        
        # 세그먼트 추출
        segment_list = []
        for i in range(start_point,(end_point-window_size), int(window_size/overlap_facor) ):
            y_sub = y[i:i+window_size]
            
            #Normalization
            y_sub = y_sub * 2 / math.pow(2,16)
            segment_list.append(y_sub)
        
        if len(segment_list) < time_step:
            continue
        
        for i in range(0, len(segment_list) - time_step+1, 30):
            temp = segment_list[i:i+time_step]
            Test_vector.append(temp)
            Test_label.append(label)
            
X_test = np.array(Test_vector)
Y_test = np.array(Test_label)

In [9]:
print (np.shape(X_test),np.shape(Y_test))

(593, 32, 1000) (593, 4)


- Arranging the data

In [10]:
X_train = np.expand_dims(X_train,3)
X_test = np.expand_dims(X_test,3)

In [11]:
random_index = np.arange(np.shape(X_train)[0])
np.random.shuffle(random_index)

In [12]:
X_train = X_train[random_index]
Y_train = Y_train[random_index]

In [13]:
print (np.shape(X_train), np.shape(X_test))

(1522, 32, 1000, 1) (593, 32, 1000, 1)


In [14]:
print (np.shape(Y_train), np.shape(Y_test))

(1522, 4) (593, 4)


# 3. Constructing the model

In [15]:
#Filter 1 initialize
filters1 = 100
kernel_size1 = 10
stride1 = 1
#Filter 2 initialize
filters2 = 160
kernel_size2 = 10
stride2 = 1

X = tf.placeholder(tf.float32, shape = [None, time_step, window_size,1])
Y = tf.placeholder(tf.float32, shape = [None, class_size])

filter1 = tf.Variable(tf.random_normal([kernel_size1,1,filters1], stddev= 0.01))
filter2 = tf.Variable(tf.random_normal([kernel_size1,filters1,filters1], stddev= 0.01))

filter3 = tf.Variable(tf.random_normal([kernel_size2,filters1,filters2], stddev= 0.01))
filter4 = tf.Variable(tf.random_normal([kernel_size2,filters2,filters2], stddev= 0.01))

stack = []
for i in range(time_step):
    conv_layer1 = tf.nn.relu(tf.nn.conv1d(X[:,i,:,:], filter1, stride1, 'VALID'))
    conv_layer2 = tf.nn.relu(tf.nn.conv1d(conv_layer1, filter2, stride2, 'VALID'))
    pooling_layer1 = tf.nn.pool(conv_layer2,[3],'MAX','SAME',strides = [3])
    
    conv_layer3 = tf.nn.relu(tf.nn.conv1d(pooling_layer1, filter3, stride2, 'VALID'))
    conv_layer4 = tf.nn.relu(tf.nn.conv1d(conv_layer3, filter4, stride2, 'VALID'))
    GAP_layer = layers.GlobalAveragePooling1D()(conv_layer4)
    #dropout = layers.Dropout(0.5)(GAP_layer)
    
    #conv_layer1 = tf.nn.relu(tf.nn.conv1d(X[:,i,:,:], filter1, stride1, 'VALID'))
    #conv_layer2 = tf.nn.relu(tf.nn.conv1d(conv_layer1, filter2, stride1, 'VALID'))
    #pooling_layer1 = tf.nn.pool(conv_layer2,[3],'MAX','SAME',strides = [3])
    #conv_layer3 = tf.nn.relu(tf.nn.conv1d(pooling_layer1, filter3, stride2, 'VALID'))
    #conv_layer4 = tf.nn.relu(tf.nn.conv1d(conv_layer3, filter4, stride2, 'VALID'))
    
    #GAP_layer = layers.GlobalAveragePooling1D()(conv_layer4)
    #dropout = tf.nn.dropout(GAP_layer, keep_prob = 0.5)
    
    stack.append(GAP_layer)
    
LSTM_input = tf.stack(stack, axis = 1)

LSTM1 = layers.LSTM(32, dropout=0.5, recurrent_dropout = 0.5, input_shape = (time_step, feature_size))(LSTM_input)

#LSTM1 = layers.LSTM(64, dropout=0.5, recurrent_dropout = 0.5, return_sequences = True, input_shape = (time_step, feature_size))(LSTM_input)
#LSTM2 = layers.LSTM(32, dropout=0.5, recurrent_dropout = 0.5 )(LSTM1)
Dense1 = layers.Dense(100, kernel_regularizer= regularizers.l2(0.001), activation='relu')(LSTM1)
dropout2 = layers.Dropout(0.5)(Dense1)

hypothesis = layers.Dense(class_size, activation='softmax')(dropout2)

In [16]:
loss = tf.reduce_mean(categorical_crossentropy(Y, hypothesis))

In [17]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

In [18]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [19]:
class BatchGenerator(object):
    def __init__(self, _X,_Y):
        self.X = _X
        self.Y = _Y
        self.cursor = 0
        self.length = len(_X)
        
    def next_batch(self, batch_size):
        flag = False
        
        if self.cursor + batch_size >= self.length:
            self.cursor = self.length - batch_size
            flag = True
            
        batch_x = self.X[self.cursor:self.cursor+batch_size]
        batch_y = self.Y[self.cursor:self.cursor+batch_size]
        
        if flag == True:
            self.cursor = 0
        else:
            self.cursor = self.cursor + batch_size
        
        return (batch_x,batch_y)

In [20]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True

In [21]:
init = tf.global_variables_initializer()
epochs = 100
batch_size = 50
train_len = len(X_train)
test_len = len(X_test)

TrainBatchGen = BatchGenerator(X_train,Y_train)
TestBatchGen = BatchGenerator(X_test,Y_test)

train_loss = np.zeros(epochs)
train_acc = np.zeros(epochs)
test_loss = np.zeros(epochs)
test_acc = np.zeros(epochs)

with tf.Session(config=config) as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        
        avg_loss = 0.
        avg_acc = 0.
        cnt = 0
        for i in range(0,train_len,batch_size):
            batch_x, batch_y = TrainBatchGen.next_batch(batch_size)
            
            print ("[%d/%d] : %.2f %% "%(i,train_len, i/train_len), end = "\r")
            
            sess.run(train, feed_dict = {X: batch_x, Y: batch_y})
            loss_ = sess.run(loss, feed_dict= {X:batch_x, Y: batch_y})
            acc_ = sess.run(accuracy, feed_dict= {X:batch_x, Y: batch_y})
            
            avg_loss += loss_
            avg_acc += acc_
            cnt += 1
            
        avg_loss = avg_loss / cnt
        avg_acc = avg_acc / cnt
        
        train_loss[epoch] = avg_loss
        train_acc[epoch] = avg_acc
        
        avg_loss = 0.
        avg_acc = 0.        
        cnt = 0
        for i in range(0,test_len,batch_size):
            batch_x, batch_y = TestBatchGen.next_batch(batch_size)
            acc_ = sess.run(accuracy, feed_dict= {X:batch_x, Y: batch_y})
            loss_ = sess.run(loss, feed_dict= {X:batch_x, Y: batch_y})
            
            avg_acc += acc_
            avg_loss += loss_
            cnt +=1
        
        avg_loss = avg_loss / cnt
        avg_acc = avg_acc / cnt
        
        test_loss[epoch] = avg_loss
        test_acc[epoch] = avg_acc
        
        print("[epoch %d/%d]: loss = %f\ttrain accuracy : %f\ttest loss = %f\ttest accuracy : %f\n" % 
              (epoch,epochs, train_loss[epoch], train_acc[epoch], test_loss[epoch], test_acc[epoch]))

[epoch 0/100]: loss = 1.230123	train accuracy : 0.425806	test loss = 1.180839	test accuracy : 0.516667

[epoch 1/100]: loss = 1.111567	train accuracy : 0.483871	test loss = 1.128577	test accuracy : 0.548333

[epoch 2/100]: loss = 1.043717	train accuracy : 0.518065	test loss = 1.152420	test accuracy : 0.546667

[epoch 3/100]: loss = 0.998596	train accuracy : 0.541935	test loss = 1.113705	test accuracy : 0.591667

[epoch 4/100]: loss = 0.952428	train accuracy : 0.558710	test loss = 1.038630	test accuracy : 0.435000

[epoch 5/100]: loss = 0.897929	train accuracy : 0.577419	test loss = 1.068841	test accuracy : 0.550000

[epoch 6/100]: loss = 0.862295	train accuracy : 0.586452	test loss = 0.980873	test accuracy : 0.576667

[epoch 7/100]: loss = 0.787217	train accuracy : 0.641935	test loss = 1.071977	test accuracy : 0.580000

[epoch 8/100]: loss = 0.796642	train accuracy : 0.618710	test loss = 0.966589	test accuracy : 0.605000

[epoch 9/100]: loss = 0.721683	train accuracy : 0.669677	test lo

KeyboardInterrupt: 